In [1]:
import pandas as pd

# CSV 파일 로드
file_path = './재정렬된_데이터.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# 주소 중복 확인
duplicated_addresses = df[df.duplicated(subset='주소', keep=False)]

if not duplicated_addresses.empty:
    print("중복된 주소가 발견되었습니다:")
    print(duplicated_addresses)
else:
    print("중복된 주소가 없습니다.")

중복된 주소가 없습니다.


In [2]:
import pandas as pd
import cx_Oracle

# 데이터 로드
file_path = './재정렬된_데이터.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 테이블에 데이터를 삽입하는 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO AI_LEARN (
                ADDRESS, PARK_NAME, PARK_DIST,
                HOS_NAME, HOS_DIST,
                SUB_NAME, SUB_DIST,
                BUS_NAME, BUS_DIST,
                PARKING_NAME, PARKING_DIST,
                LOW_NAME, LOW_DIST,
                MID_NAME, MID_DIST,
                HI_NAME, HI_DIST
            ) VALUES (
                :1, :2, :3, 
                :4, :5, 
                :6, :7, 
                :8, :9, 
                :10, :11, 
                :12, :13, 
                :14, :15, 
                :16, :17
            )
        """
        cursor.execute(query, (
            row['주소'], 
            row['공원_요소명'], 
            row['공원_거리 (m)'], 
            row['기관명_요소명'], 
            row['기관명_거리 (m)'], 
            row['역명_요소명'], 
            row['역명_거리 (m)'], 
            row['정류장명_요소명'], 
            row['정류장명_거리 (m)'], 
            row['주차장_요소명'], 
            row['주차장_거리 (m)'], 
            row['초등학교명'], 
            row['초등학교 거리'], 
            row['중학교명'], 
            row['중학교 거리'], 
            row['고등학교명'], 
            row['고등학교 거리']
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 인해 삽입 실패: {row['주소']}")
    except Exception as e:
        print(f"오류 발생: {e}")

# DataFrame의 각 행 데이터를 삽입
for _, row in df.iterrows():
    insert_data(row)

# DB에 커밋
connection.commit()

# 연결 종료
cursor.close()
connection.close()

print("데이터 삽입 완료")


데이터 삽입 완료


In [3]:
import pandas as pd
import cx_Oracle

# 데이터 로드
file_path = './서울시 주차장 최종.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')
df.head()

,주차장명,위도,경도,주소,전화번호,유무료구분,기본 주차 요금,추가 단위 요금,기본 주차 시간(분 단위),추가 단위 시간(분 단위),일 최대 요금,버스 기본 주차 요금,버스 추가 단위 요금,주소_정제
0,영등포여고 공영(구),37.515205,126.915116,영등포구 신길동 184-3,02-2677-1401,Y,50.0,50.0,5.0,5.0,0.0,0.0,0.0,영등포구 신길동 184-3
1,휘경마을 공영주차장(구),37.588069,127.058093,동대문구 휘경동 308-32,02-2247-9664~6,Y,1000.0,0.0,60.0,0.0,10000.0,0.0,0.0,동대문구 휘경동 308-32
2,압구정 428 공영주차장(구),37.527017,127.026471,강남구 압구정동 428-0,02-2176-0971,Y,400.0,400.0,5.0,5.0,NaN,0.0,0.0,강남구 압구정동 428-0
3,포이초교 공영주차장(구),37.475646,127.052365,강남구 개포동 1273-0,02-2176-0925,Y,200.0,200.0,5.0,5.0,NaN,0.0,0.0,강남구 개포동 1273-0
4,구로디지털단지역 공영주차장(시),37.485432,126.901243,구로구 구로동 810-3,02-2290-6034,Y,260.0,260.0,5.0,5.0,18700.0,0.0,0.0,구로구 구로동 810-3


In [4]:
import pandas as pd
import cx_Oracle

# 데이터 로드
file_path = './서울시 주차장 최종.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# NaN 값을 처리 (숫자 컬럼은 0, 문자열 컬럼은 빈 문자열로 대체)
df.fillna({
    '주차장명': '',  # PARKING_NAME
    '위도': 0,  # PARKING_LAT
    '경도': 0,  # PARKING_LON
    '주소': '',  # PARKING_ADDRESS
    '전화번호': '',  # PARKING_CALL
    '유무료구분': '',  # PARKING_FREE
    '기본 주차 요금': 0,  # PARKING_PLANE
    '추가 단위 요금': 0,  # PARKING_MORE
    '기본 주차 시간(분 단위)': 0,  # PARKING_PLANE_TIME
    '추가 단위 시간(분 단위)': 0,  # PARKING_MORE_TIME
    '일 최대 요금': 0,  # PARKING_DMAX
    '버스 기본 주차 요금': 0,  # PARKING_BUS_PLANE
    '버스 추가 단위 요금': 0  # PARKING_BUS_MORE
}, inplace=True)

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 테이블에 데이터를 삽입하는 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO PARKING (
                PARKING_NAME, PARKING_LAT, PARKING_LON, PARKING_ADDRESS,
                PARKING_CALL, PARKING_FREE, PARKING_PLANE, PARKING_MORE,
                PARKING_PLANE_TIME, PARKING_MORE_TIME, PARKING_DMAX,
                PARKING_BUS_PLANE, PARKING_BUS_MORE
            ) VALUES (
                :1, :2, :3, :4,
                :5, :6, :7, :8,
                :9, :10, :11, :12, :13
            )
        """
        cursor.execute(query, (
            row['주차장명'], 
            row['위도'], 
            row['경도'], 
            row['주소'], 
            row['전화번호'], 
            row['유무료구분'], 
            row['기본 주차 요금'], 
            row['추가 단위 요금'], 
            row['기본 주차 시간(분 단위)'], 
            row['추가 단위 시간(분 단위)'], 
            row['일 최대 요금'], 
            row['버스 기본 주차 요금'], 
            row['버스 추가 단위 요금']
        ))

    except cx_Oracle.IntegrityError as e:
        print(f"중복 데이터 삽입 오류: {row['주차장명']}, 오류 메시지: {e}")
    except Exception as e:
        print(f"오류 발생: {e}, 데이터: {row}")

# DataFrame의 각 행 데이터를 삽입
for _, row in df.iterrows():
    insert_data(row)

# DB에 커밋
connection.commit()

# 연결 종료
cursor.close()
connection.close()

print("데이터 삽입 완료")


데이터 삽입 완료


In [5]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './서울 공원 결측치_제거.csv'
df = pd.read_csv(file_path, encoding='cp949')

# NaN 값을 처리 (숫자형 컬럼은 0, 문자열 컬럼은 빈 문자열로 대체)
df.fillna({
    '공원명': '',    # PARK_NAME
    '위도': 0,       # PARK_LAT
    '경도': 0,       # PARK_LON
    '이미지': '',    # PARK_IMG
    '지역': '',      # PARK_AREA
    '공원주소': ''   # PARK_ADDRESS
}, inplace=True)

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO PARK (
                PARK_NAME, PARK_LAT, PARK_LON, PARK_IMG, PARK_AREA, PARK_ADDRESS
            ) VALUES (
                :1, :2, :3, :4, :5, :6
            )
        """
        cursor.execute(query, (
            row['공원명'],         # PARK_NAME
            row['위도'],           # PARK_LAT
            row['경도'],           # PARK_LON
            row['이미지'],         # PARK_IMG
            row['지역'],           # PARK_AREA
            row['공원주소']        # PARK_ADDRESS
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['공원명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 공원명: {row['공원명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [6]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './지하철_전처리_최종.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO SUBWAY (
                SUB_NAME, SUB_LAT, SUB_LON
            ) VALUES (
                :1, :2, :3
            )
        """
        cursor.execute(query, (
            row['역명'],  # SUB_NAME
            row['위도'],  # SUB_LAT
            row['경도']   # SUB_LON
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['역명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 역명: {row['역명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [7]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './hospital.csv'
df = pd.read_csv(file_path, encoding='utf-8')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO HOSPITAL (
                HOS_NAME, HOS_LAT, HOS_LON, HOS_ADDRESS,
                HOS_CALL, HOS_S, HOS_C
            ) VALUES (
                :1, :2, :3, :4, :5, :6, :7
            )
        """
        cursor.execute(query, (
            row['기관명'],           # HOS_NAME
            row['병원위도'],         # HOS_LAT
            row['병원경도'],         # HOS_LON
            row['주소'],             # HOS_ADDRESS
            row['대표전화'],         # HOS_CALL
            row['진료시간(월요일)S'], # HOS_S
            row['진료시간(월요일)C']  # HOS_C
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['기관명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 기관명: {row['기관명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [8]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './bus_stop.csv'
df = pd.read_csv(file_path, encoding='utf-8')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO BUS (
                BUS_NAME, BUS_LAT, BUS_LON
            ) VALUES (
                :1, :2, :3
            )
        """
        cursor.execute(query, (
            row['정류장명'],  # BUS_NAME
            row['위도'],     # BUS_LAT
            row['경도']      # BUS_LON
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['정류장명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 정류장명: {row['정류장명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [9]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './초등학교.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO LOWSCHOOLE (
                LOW_NAME, LOW_LAT, LOW_LON, LOW_BUIL, LOW_ADDRESS
            ) VALUES (
                :1, :2, :3, :4, :5
            )
        """
        cursor.execute(query, (
            row['학교명'],      # LOW_NAME
            row['위도'],        # LOW_LAT
            row['경도'],        # LOW_LON
            row['설립형태'],    # LOW_BUIL
            row['소재지도로명주소']  # LOW_ADDRESS
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['학교명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 학교명: {row['학교명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [10]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './중학교.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO MIDDLESCHOOLE (
                MID_NAME, MID_LAT, MID_LON, MID_BUIL, MID_ADDRESS
            ) VALUES (
                :1, :2, :3, :4, :5
            )
        """
        cursor.execute(query, (
            row['학교명'],        # MID_NAME
            row['위도'],          # MID_LAT
            row['경도'],          # MID_LON
            row['설립형태'],      # MID_BUIL
            row['소재지도로명주소'] # MID_ADDRESS
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['학교명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 학교명: {row['학교명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [11]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './고등학교.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO HIGHSCHOOLE (
                HI_NAME, HI_LAT, HI_LON, HI_BUIL, HI_ADDRESS
            ) VALUES (
                :1, :2, :3, :4, :5
            )
        """
        cursor.execute(query, (
            row['학교명'],        # HI_NAME
            row['위도'],          # HI_LAT
            row['경도'],          # HI_LON
            row['설립형태'],      # HI_BUIL
            row['소재지도로명주소'] # HI_ADDRESS
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['학교명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 학교명: {row['학교명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [14]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './마이홈_mergefinal.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')
df.columns

Index(['공급번호', '공급면적(전용)', '공급면적(공용)', '임대보증금(원)', '월 임대료(원)', '전환보증금(원)',
       '단지명', '주소', '주택유형', '세대수', '임대사업자', '주차대수', 'img'],
      dtype='object')

In [15]:
'주소','단지명',  '주택유형', '세대수', '임대사업자', '주차대수', 'img','공급번호', '공급면적(전용)', '공급면적(공용)', '임대보증금(원)', '월 임대료(원)', '전환보증금(원)'

('주소',
 '단지명',
 '주택유형',
 '세대수',
 '임대사업자',
 '주차대수',
 'img',
 '공급번호',
 '공급면적(전용)',
 '공급면적(공용)',
 '임대보증금(원)',
 '월 임대료(원)',
 '전환보증금(원)')

In [16]:
# 결측치 처리 (NaN 값 대체)
# 숫자형 컬럼은 0으로 대체, 문자열 컬럼은 빈 문자열로 대체
df.fillna({
    '주소': '',                 # HOME_ADDRESS (VARCHAR2)
    '단지명': '',               # HOME_NAME (VARCHAR2)
    '주택유형': '',             # HOME_KIND (VARCHAR2)
    '세대수': 0,                # HOME_COUNT (INTEGER)
    '임대사업자': '',           # HOME_CO (VARCHAR2)
    '주차대수': 0,              # HOME_PARKING (INTEGER)
    'img': '',                  # HOME_IMG (VARCHAR2)
    '공급번호': 0,              # HOME_NO (INTEGER)
    '공급면적(전용)': '0',       # HOME_MYAREA (VARCHAR2)
    '공급면적(공용)': '0',       # HOME_WEAREA (VARCHAR2)
    '임대보증금(원)': '0',       # HOME_DEP (VARCHAR2)
    '월 임대료(원)': '0',        # HOME_MOTH_PAI (VARCHAR2)
    '전환보증금(원)': '0'        # HOME_CHANGE_DEP (VARCHAR2)
}, inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52352 entries, 0 to 52351
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   공급번호      52352 non-null  float64
 1   공급면적(전용)  52352 non-null  object 
 2   공급면적(공용)  52352 non-null  object 
 3   임대보증금(원)  52352 non-null  object 
 4   월 임대료(원)  52352 non-null  object 
 5   전환보증금(원)  52352 non-null  object 
 6   단지명       52352 non-null  object 
 7   주소        52352 non-null  object 
 8   주택유형      52352 non-null  object 
 9   세대수       52352 non-null  object 
 10  임대사업자     52352 non-null  object 
 11  주차대수      52352 non-null  object 
 12  img       52352 non-null  object 
dtypes: float64(1), object(12)
memory usage: 5.2+ MB


In [2]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './마이홈_mergefinal.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')
df.head()

,공급번호,공급면적(전용),공급면적(공용),임대보증금(원),월 임대료(원),전환보증금(원),단지명,주소,주택유형,세대수,임대사업자,주차대수,img
0,45.0,45.73㎡,3.06㎡,"4,951,000","338,610",0,서울특별시 강남구,서울특별시 강남구 논현로117길 12-4,다가구주택,7,LH서울,0,https://www.myhome.go.kr/jfile/preview.do?file...
1,46.0,46.89㎡,3.14㎡,"5,104,000","343,230",0,서울특별시 강남구,서울특별시 강남구 논현로117길 12-4,다가구주택,7,LH서울,0,https://www.myhome.go.kr/jfile/preview.do?file...
2,49.0,49.88㎡,3.34㎡,"3,606,000","199,330",0,서울특별시 강남구,서울특별시 강남구 논현로117길 12-4,다가구주택,7,LH서울,0,https://www.myhome.go.kr/jfile/preview.do?file...
3,49.0,49.93㎡,3.34㎡,"5,467,000","384,330",0,서울특별시 강남구,서울특별시 강남구 논현로117길 12-4,다가구주택,7,LH서울,0,https://www.myhome.go.kr/jfile/preview.do?file...
4,51.0,51.62㎡,3.46㎡,"3,742,000","206,230",0,서울특별시 강남구,서울특별시 강남구 논현로117길 12-4,다가구주택,7,LH서울,0,https://www.myhome.go.kr/jfile/preview.do?file...


In [3]:
import pandas as pd

# CSV 파일 불러오기
file_path1 = './마이홈_주소_위도경도_정리.csv'
file_path2 = './마이홈_mergefinal.csv'

df_geo = pd.read_csv(file_path1, encoding='utf-8-sig')
df_main = pd.read_csv(file_path2, encoding='utf-8-sig')

# 병합: 주소를 기준으로 left join
merged_df = pd.merge(df_main, df_geo[['주소', '위도', '경도']], on='주소', how='left')

# 결과 확인
print(merged_df.head())

# 병합 결과를 새로운 CSV 파일로 저장
merged_df.to_csv('./마이홈_merged_with_geo.csv', index=False, encoding='utf-8-sig')


   공급번호 공급면적(전용) 공급면적(공용)   임대보증금(원) 월 임대료(원) 전환보증금(원)        단지명  \
0  45.0   45.73㎡    3.06㎡  4,951,000  338,610        0  서울특별시 강남구   
1  46.0   46.89㎡    3.14㎡  5,104,000  343,230        0  서울특별시 강남구   
2  49.0   49.88㎡    3.34㎡  3,606,000  199,330        0  서울특별시 강남구   
3  49.0   49.93㎡    3.34㎡  5,467,000  384,330        0  서울특별시 강남구   
4  51.0   51.62㎡    3.46㎡  3,742,000  206,230        0  서울특별시 강남구   

                       주소   주택유형 세대수 임대사업자 주차대수  \
0  서울특별시 강남구 논현로117길 12-4  다가구주택   7  LH서울    0   
1  서울특별시 강남구 논현로117길 12-4  다가구주택   7  LH서울    0   
2  서울특별시 강남구 논현로117길 12-4  다가구주택   7  LH서울    0   
3  서울특별시 강남구 논현로117길 12-4  다가구주택   7  LH서울    0   
4  서울특별시 강남구 논현로117길 12-4  다가구주택   7  LH서울    0   

                                                 img         위도          경도  
0  https://www.myhome.go.kr/jfile/preview.do?file...  37.509894  127.031402  
1  https://www.myhome.go.kr/jfile/preview.do?file...  37.509894  127.031402  
2  https://www.myhome.go.kr/jfile/preview.do

In [6]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './마이홈_merged_with_geo.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')
df.columns

Index(['공급번호', '공급면적(전용)', '공급면적(공용)', '임대보증금(원)', '월 임대료(원)', '전환보증금(원)',
       '단지명', '주소', '주택유형', '세대수', '임대사업자', '주차대수', 'img', '위도', '경도'],
      dtype='object')

In [12]:
import pandas as pd
import cx_Oracle

# CSV 파일 로드
file_path = './마이홈_merged_with_geo.csv'
df = pd.read_csv(file_path, encoding='utf-8-sig')

# Oracle DB 연결 정보
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="XE")
connection = cx_Oracle.connect(user="C##SCOTT", password="tiger", dsn=dsn)

# Cursor 생성
cursor = connection.cursor()

# 결측치 처리 (NaN 값 대체)
# 숫자형 컬럼은 0으로 대체, 문자열 컬럼은 빈 문자열로 대체
df.fillna({
    '주소': '',                 # HOME_ADDRESS (VARCHAR2)
    '단지명': '',               # HOME_NAME (VARCHAR2)
    '주택유형': '',             # HOME_KIND (VARCHAR2)
    '세대수': '',                # HOME_COUNT (INTEGER)
    '임대사업자': '',           # HOME_CO (VARCHAR2)
    '주차대수': '',              # HOME_PARKING (INTEGER)
    'img': '',                  # HOME_IMG (VARCHAR2)
    '공급번호': 0,              # HOME_NO (INTEGER)
    '공급면적(전용)': '0',       # HOME_MYAREA (VARCHAR2)
    '공급면적(공용)': '0',       # HOME_WEAREA (VARCHAR2)
    '임대보증금(원)': '0',       # HOME_DEP (VARCHAR2)
    '월 임대료(원)': '0',        # HOME_MOTH_PAI (VARCHAR2)
    '전환보증금(원)': '0',        # HOME_CHANGE_DEP (VARCHAR2)
    '위도': 0,
    '경도': 0
}, inplace=True)

# 데이터 삽입 함수
def insert_data(row):
    try:
        query = """
            INSERT INTO HOME (
                HOME_ADDRESS, HOME_LAT, HOME_LON, HOME_NAME, HOME_KIND, HOME_COUNT, HOME_CO,
                HOME_PARKING, HOME_IMG, HOME_NO, HOME_MYAREA, HOME_WEAREA,
                HOME_DEP, HOME_MOTH_PAI, HOME_CHANGE_DEP, HOME_RECR
            ) VALUES (
                :1, :2, :3, :4, :5, 
                :6, :7, :8, :9, :10,
                :11, :12, :13, :14, :15, 'N'
            )
        """
        cursor.execute(query, (
            row['주소'],             # HOME_ADDRESS
            row['위도'],             # HOME_ADDRESS
            row['경도'],             # HOME_ADDRESS
            row['단지명'],           # HOME_NAME
            row['주택유형'],         # HOME_KIND
            row['세대수'],           # HOME_COUNT
            row['임대사업자'],       # HOME_CO
            row['주차대수'],         # HOME_PARKING
            row['img'],             # HOME_IMG
            row['공급번호'],         # HOME_NO
            row['공급면적(전용)'],   # HOME_MYAREA
            row['공급면적(공용)'],   # HOME_WEAREA
            row['임대보증금(원)'],   # HOME_DEP
            row['월 임대료(원)'],    # HOME_MOTH_PAI
            row['전환보증금(원)']    # HOME_CHANGE_DEP
        ))
    except cx_Oracle.IntegrityError:
        print(f"중복 데이터로 삽입 실패: {row['주소']}, 단지명: {row['단지명']}")
    except Exception as e:
        print(f"오류 발생: {e}, 주소: {row['주소']}, 단지명: {row['단지명']}")

# 데이터 삽입 실행
for _, row in df.iterrows():
    insert_data(row)

# 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("데이터 삽입 완료!")


중복 데이터로 삽입 실패: 서울특별시 성북구 인촌로12길 34, 단지명: 34
중복 데이터로 삽입 실패: 서울특별시 성북구 인촌로12길 34, 단지명: 34
중복 데이터로 삽입 실패: 서울특별시 성북구 인촌로12길 34, 단지명: 34
중복 데이터로 삽입 실패: 서울특별시 양천구 신월로27길 12, 단지명: 동원데자뷰
중복 데이터로 삽입 실패: 서울특별시 양천구 신월로27길 12, 단지명: 동원데자뷰
중복 데이터로 삽입 실패: 서울특별시 광진구 아차산로28길 6-5, 단지명: 아차산로28길 6-5
중복 데이터로 삽입 실패: 서울특별시 광진구 아차산로28길 6-5, 단지명: 아차산로28길 6-5
중복 데이터로 삽입 실패: 서울특별시 광진구 아차산로28길 6-5, 단지명: 아차산로28길 6-5
중복 데이터로 삽입 실패: 서울특별시 광진구 아차산로28길 6-5, 단지명: 아차산로28길 6-5
데이터 삽입 완료!
